## ReAct Design Pattern Simple Demo

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv("GOOGLE_API_KEY")

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.2  #more deterministic
)

### Creating Simple Tools

In [ ]:
import datetime

def get_current_time():
    print('Current Time Tool Called')
    return datetime.datetime.now().strftime("%H:%M:%S")

def get_current_date():
    print('Current Date Tool Called')
    return datetime.datetime.now().strftime("%Y-%m-%d")

In [ ]:
SYSTEM_PROMPT = """You are a ReAct (Reasoning + Act) style agent.

Your available tools are:
- get_current_date: Gets the current date (No arguments)
- get_current_time: Gets the current time (No arguments)

Rules:
- Think step-by-step internally, but DO NOT reveal your internal thoughts.
- When you need a tool, respond EXACTLY in this format:
  ACTION: <tool_name> | <tool_input>
- When you are ready to answer the user, respond EXACTLY:
  FINAL: <your answer>
"""

In [ ]:
TOOLS = {
    "get_current_date": get_current_date,
    "get_current_time": get_current_time
}

### ReAct Loop

In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

In [ ]:
# Simple function with no error catching, for demo purposes
def run_react(question, max_steps=6):
    messages = [SystemMessage(content=SYSTEM_PROMPT), HumanMessage(content=question)]

    for step in range(max_steps):
        resp = llm.invoke(messages).content.strip()
        print(f"\nStep {step+1}: {resp}")

        if resp.startswith("FINAL:"):
            return resp[len("FINAL:"):].strip()

        if resp.startswith("ACTION:"):
            _, rest = resp.split("ACTION:", 1)
            tool_part= rest.split("|")
            tool_name = tool_part[0].strip()
            tool_output = TOOLS[tool_name]() if tool_name in TOOLS else f"Unknown tool: {tool_name}"
            print(f"Observation: {tool_output}")
            messages.append(AIMessage(content=resp))
            messages.append(HumanMessage(content=f"OBSERVATION: {tool_output}"))

    return "max_steps reached without FINAL."

In [ ]:
answer = run_react("What date is today?")
print("\nFinal Answer:", answer)